## Evaluation of Cross Dataset Performance

In [1]:
import os
import torch
import cv2
from bunch import Bunch
from ruamel.yaml import safe_load
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from PIL import Image

import torch.backends.cudnn as cudnn
from glob import glob
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc


from tester import Tester
from utils import losses
from utils.helpers import get_instance

from datasets.utils import clahe_equalized
from datasets.transform import pipeline_tranforms
import networks as models


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Dataset Class

In [3]:
class CHASEDBDataset(Dataset):

    def __init__(self, data_path):
        self.images_path = sorted(glob(data_path + "/test_data/*"))
        self.masks_path = sorted(glob(data_path + "/test_label/*"))

        self.transforms = None
        self.size = 512
        self.n_samples = len(self.images_path)

    def __getitem__(self, index):
        """ Reading image """
        image = cv2.imread(self.images_path[index], cv2.IMREAD_COLOR)
        image = clahe_equalized(image)
        image = cv2.resize(image, (self.size, self.size), interpolation=cv2.INTER_NEAREST)

        image = image / 255.0  # (512, 512, 3) Normalizing to range (0,1)
        image = np.transpose(image, (2, 0, 1))  # (3, 512, 512)
        image = image.astype(np.float32)
        image = torch.from_numpy(image)

        """ Reading mask """
        try:
            mask = cv2.imread(self.masks_path[index], cv2.IMREAD_GRAYSCALE)
            # print(self.masks_path[index])
            mask = cv2.resize(mask, (self.size, self.size), interpolation=cv2.INTER_NEAREST)
        except Exception as e:
            print(str(e))
            print(self.masks_path[index])
        
        mask = mask / 255.0  # (512, 512)
        mask = np.expand_dims(mask, axis=0)  # (1, 512, 512)
        mask = mask.astype(np.float32)
        mask = torch.from_numpy(mask)

        return image, mask

    def __len__(self):
        return self.n_samples
    

chase_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/datasets/CHASE_DB1'
chase_dataset = CHASEDBDataset(chase_path)

chase_dataset.__len__()

8

In [4]:
class DRIVEDataset(Dataset):

    def __init__(self, data_path):
        super().__init__()
        
        self.images_path = sorted(glob(data_path + "/test_data/*"))
        self.masks_path = sorted(glob(data_path + "/test_label/*"))
        self.n_samples = len(self.images_path)

    def __getitem__(self, index):

        data = Image.open(self.images_path[index]).resize((512,512), resample=Image.Resampling.NEAREST)
        label = Image.open(self.masks_path[index]).resize((512,512), resample=Image.Resampling.NEAREST)

        data = np.array(data)
        label = np.array(label)

        if data.shape[-1]==3:
            data = torch.from_numpy(np.array(data).transpose(2, 0, 1)).float() / 255
            label = torch.from_numpy(np.array(label)).float().unsqueeze(0) / 255
        else:
            data = torch.from_numpy(data).unsqueeze(0).float() / 255
            label = torch.from_numpy(label).float().unsqueeze(0) / 255

        return data, label
    
    def __len__(self):
        return self.n_samples

drive_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/datasets/DRIVE'
drive_dataset = DRIVEDataset(drive_path)

drive_dataset.__len__()

20

In [5]:
class FIVES(Dataset):

    def __init__(self, data_path):
        super(FIVES, self).__init__()
        self.transforms = pipeline_tranforms()
        
        self.images_path = sorted(glob(data_path + "/test/Original/*"))
        self.masks_path  = sorted(glob(data_path + "/test/Ground truth/*"))

        self.n_samples = len(self.images_path)
    
    def __getitem__(self, index):
        """ Reading image """
        image = cv2.imread(self.images_path[index], cv2.IMREAD_COLOR)
        image = clahe_equalized(image)
        image = cv2.resize(image, (512, 512), interpolation=cv2.INTER_NEAREST)

        image = image / 255.0  # type: ignore # (512, 512, 3) Normalizing to range (0,1)
        image = np.transpose(image, (2, 0, 1))  # (3, 512, 512)
        image = image.astype(np.float32)
        image = torch.from_numpy(image)

        """ Reading mask """
        mask = cv2.imread(self.masks_path[index], cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask,  (512, 512), interpolation=cv2.INTER_NEAREST)
        mask = mask / 255.0  # type: ignore # (512, 512)
        mask = np.expand_dims(mask, axis=0)  # (1, 512, 512)
        mask = mask.astype(np.float32)
        mask = torch.from_numpy(mask)
        
        return image, mask

    def __len__(self):
        return self.n_samples
fives_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/datasets/FIVES'

fives_dataset = FIVES(fives_path)
fives_dataset.__len__()

200

In [7]:
def main_result(config_file, dataset, bs, weight_path):

    with open(config_file, encoding="utf-8") as file:
        CFG = Bunch(safe_load(file))
        
    checkpoint = torch.load(weight_path)
    CFG_ck = checkpoint['config']

    test_loader = DataLoader(dataset, bs ,shuffle=False,  
                            num_workers=CFG.num_workers, pin_memory=True)
    
    model = get_instance(models, 'model', CFG)
    loss = get_instance(losses, 'loss', CFG_ck)

    test = Tester(model, loss, CFG, checkpoint, test_loader, device)
    test.test()

## WNET 

In [7]:
# FIVES on FIVES 
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/WNet/FIVES/CE_DiceLoss/checkpoint-epoch100.pth'
config_file = '../configs/lwnet_cldice.yaml'

main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

# Fives on Chase

main_result(config_file, dataset=chase_dataset, bs=2, weight_path=weight_path)
# Fives on Drive

main_result(config_file, dataset=drive_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.2648 | AUC 0.9865 F1 0.8565 Acc 0.9806  Sen 0.8310 Spe 0.9915 Pre 0.8869 IOU 0.7508 MCC 0.8474 |B 0.24 D 0.08 |: 100%|██████████| 50/50 [00:43<00:00,  1.16it/s]
2023-11-17 03:13:40.741 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-11-17 03:13:40.742 | INFO     | tester:test:79 - test time:  0.2431
2023-11-17 03:13:40.742 | INFO     | tester:test:80 -      loss:  0.2648
2023-11-17 03:13:40.743 | INFO     | tester:test:84 - AUC  : 0.9865
2023-11-17 03:13:40.744 | INFO     | tester:test:84 - F1   : 0.8565
2023-11-17 03:13:40.744 | INFO     | tester:test:84 - Acc  : 0.9806
2023-11-17 03:13:40.744 | INFO     | tester:test:84 - Sen  : 0.831
2023-11-17 03:13:40.745 | INFO     | tester:test:84 - Spe  : 0.9915
2023-11-17 03:13:40.745 | INFO     | tester:test:84 - pre  : 0.8869
2023-11-17 03:13:40.746 | INFO     | tester:test:84 - IOU  : 0.7508
2023-11-17 03:13:40.746 | INFO     | tester:test:84 - MCC  : 0.8474
TEST (3) | Loss: 0.5900 | AUC 0.9723 F1 0.7221

In [8]:
# CHASEDB on CHASEDB
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/WNet/CHASE/CE_DiceLoss/checkpoint-epoch80.pth'
config_file = "../configs/lwnet_dicebce.yaml"


main_result(config_file, dataset=chase_dataset, bs=2, weight_path=weight_path)

# Chase on FIVES
main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

# Chase on DRIVE
main_result(config_file, dataset=drive_dataset, bs=4, weight_path=weight_path)

  0%|          | 0/4 [00:00<?, ?it/s]

TEST (3) | Loss: 0.6387 | AUC 0.9630 F1 0.7499 Acc 0.9672  Sen 0.7797 Spe 0.9798 Pre 0.7243 IOU 0.6004 MCC 0.7336 |B 0.19 D 0.17 |: 100%|██████████| 4/4 [00:01<00:00,  2.05it/s]
2023-11-17 03:13:51.795 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-11-17 03:13:51.796 | INFO     | tester:test:79 - test time:  0.1901
2023-11-17 03:13:51.797 | INFO     | tester:test:80 -      loss:  0.6387
2023-11-17 03:13:51.797 | INFO     | tester:test:84 - AUC  : 0.963
2023-11-17 03:13:51.798 | INFO     | tester:test:84 - F1   : 0.7499
2023-11-17 03:13:51.798 | INFO     | tester:test:84 - Acc  : 0.9672
2023-11-17 03:13:51.799 | INFO     | tester:test:84 - Sen  : 0.7797
2023-11-17 03:13:51.800 | INFO     | tester:test:84 - Spe  : 0.9798
2023-11-17 03:13:51.800 | INFO     | tester:test:84 - pre  : 0.7243
2023-11-17 03:13:51.801 | INFO     | tester:test:84 - IOU  : 0.6004
2023-11-17 03:13:51.801 | INFO     | tester:test:84 - MCC  : 0.7336
TEST (49) | Loss: 0.6686 | AUC 0.9279 F1 0.7033 A

In [9]:
# DRIVE on DRIVE
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/WNet/DRIVE/CE_DiceLoss/checkpoint-epoch80.pth'
config_file = "../configs/lwnet_dicebce.yaml"


main_result(config_file, dataset=drive_dataset, bs=4, weight_path=weight_path)

# DRIVE on CHASE
main_result(config_file, dataset=chase_dataset, bs=2, weight_path=weight_path)

# DRIVE on FIVES
main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)



  0%|          | 0/5 [00:00<?, ?it/s]

TEST (4) | Loss: 0.5552 | AUC 0.9653 F1 0.7798 Acc 0.9622  Sen 0.7662 Spe 0.9811 Pre 0.7961 IOU 0.6391 MCC 0.7599 |B 1.83 D 1.81 |: 100%|██████████| 5/5 [00:12<00:00,  2.45s/it]
2023-11-17 03:14:46.846 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-11-17 03:14:46.846 | INFO     | tester:test:79 - test time:  1.8338
2023-11-17 03:14:46.847 | INFO     | tester:test:80 -      loss:  0.5552
2023-11-17 03:14:46.847 | INFO     | tester:test:84 - AUC  : 0.9653
2023-11-17 03:14:46.848 | INFO     | tester:test:84 - F1   : 0.7798
2023-11-17 03:14:46.848 | INFO     | tester:test:84 - Acc  : 0.9622
2023-11-17 03:14:46.849 | INFO     | tester:test:84 - Sen  : 0.7662
2023-11-17 03:14:46.849 | INFO     | tester:test:84 - Spe  : 0.9811
2023-11-17 03:14:46.850 | INFO     | tester:test:84 - pre  : 0.7961
2023-11-17 03:14:46.850 | INFO     | tester:test:84 - IOU  : 0.6391
2023-11-17 03:14:46.851 | INFO     | tester:test:84 - MCC  : 0.7599
TEST (3) | Loss: 0.9662 | AUC 0.9603 F1 0.6493 A

## SA-UNet

In [11]:
# CHASEDB on CHASEDB
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/SA_UNet/CHASE/CE_DiceLoss/checkpoint-epoch150.pth'
config_file = "../configs/sa_unet_dicebce.yaml"


main_result(config_file, dataset=chase_dataset, bs=4, weight_path=weight_path)

TEST (1) | Loss: 0.1600 | AUC 0.9884 F1 0.7930 Acc 0.9708  Sen 0.8877 Spe 0.9764 Pre 0.7172 IOU 0.6569 MCC 0.7828 |B 0.48 D 0.39 |: 100%|██████████| 2/2 [00:02<00:00,  1.13s/it]
2023-10-15 21:20:57.489 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-10-15 21:20:57.490 | INFO     | tester:test:79 - test time:  0.4827
2023-10-15 21:20:57.491 | INFO     | tester:test:80 -      loss:  0.16
2023-10-15 21:20:57.491 | INFO     | tester:test:84 - AUC  : 0.9884
2023-10-15 21:20:57.492 | INFO     | tester:test:84 - F1   : 0.793
2023-10-15 21:20:57.492 | INFO     | tester:test:84 - Acc  : 0.9708
2023-10-15 21:20:57.493 | INFO     | tester:test:84 - Sen  : 0.8877
2023-10-15 21:20:57.493 | INFO     | tester:test:84 - Spe  : 0.9764
2023-10-15 21:20:57.493 | INFO     | tester:test:84 - pre  : 0.7172
2023-10-15 21:20:57.495 | INFO     | tester:test:84 - IOU  : 0.6569
2023-10-15 21:20:57.495 | INFO     | tester:test:84 - MCC  : 0.7828


In [13]:
# CHASEDB on DRIVE
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/SA_UNet/CHASE/CE_DiceLoss/checkpoint-epoch150.pth'
config_file = "../config/sa_unet_chase.yaml"


main_result(config_file, dataset=drive_dataset, bs=4, weight_path=weight_path)

# CHASEDB on FIVES
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/SA_UNet/CHASE/CE_DiceLoss/checkpoint-epoch150.pth'
config_file = "../config/sa_unet_chase.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

  0%|          | 0/5 [00:00<?, ?it/s]

TEST (4) | Loss: 0.2090 | AUC 0.9570 F1 0.7527 Acc 0.9602  Sen 0.6923 Spe 0.9861 Pre 0.8268 IOU 0.6038 MCC 0.7352 |B 0.21 D 0.19 |: 100%|██████████| 5/5 [00:04<00:00,  1.20it/s]
2023-10-15 21:23:27.027 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-10-15 21:23:27.028 | INFO     | tester:test:79 - test time:  0.2086
2023-10-15 21:23:27.029 | INFO     | tester:test:80 -      loss:  0.209
2023-10-15 21:23:27.029 | INFO     | tester:test:84 - AUC  : 0.957
2023-10-15 21:23:27.030 | INFO     | tester:test:84 - F1   : 0.7527
2023-10-15 21:23:27.030 | INFO     | tester:test:84 - Acc  : 0.9602
2023-10-15 21:23:27.031 | INFO     | tester:test:84 - Sen  : 0.6923
2023-10-15 21:23:27.032 | INFO     | tester:test:84 - Spe  : 0.9861
2023-10-15 21:23:27.033 | INFO     | tester:test:84 - pre  : 0.8268
2023-10-15 21:23:27.033 | INFO     | tester:test:84 - IOU  : 0.6038
2023-10-15 21:23:27.034 | INFO     | tester:test:84 - MCC  : 0.7352
TEST (49) | Loss: 0.1555 | AUC 0.9806 F1 0.8050 Ac

In [15]:
# DRIVE On DRIVE

weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/SA_UNet/DRIVE/CE_DiceLoss/checkpoint-epoch150.pth'
config_file = "../config/sa_unet_drive.yaml"


main_result(config_file, dataset=drive_dataset, bs=4, weight_path=weight_path)

# Drive on Chase
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/SA_UNet/DRIVE/CE_DiceLoss/checkpoint-epoch150.pth'
config_file = "../config/sa_unet_drive.yaml"


main_result(config_file, dataset=chase_dataset, bs=2, weight_path=weight_path)

# Drive on FIVES
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/SA_UNet/DRIVE/CE_DiceLoss/checkpoint-epoch150.pth'
config_file = "../config/sa_unet_drive.yaml"


main_result(config_file, dataset=fives_dataset, bs=4,  weight_path=weight_path)

  0%|          | 0/5 [00:00<?, ?it/s]

TEST (4) | Loss: 0.1512 | AUC 0.9850 F1 0.8165 Acc 0.9676  Sen 0.8249 Spe 0.9814 Pre 0.8107 IOU 0.6900 MCC 0.7995 |B 0.33 D 0.31 |: 100%|██████████| 5/5 [00:04<00:00,  1.05it/s]
2023-10-15 21:40:22.768 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-10-15 21:40:22.768 | INFO     | tester:test:79 - test time:  0.327
2023-10-15 21:40:22.769 | INFO     | tester:test:80 -      loss:  0.1512
2023-10-15 21:40:22.770 | INFO     | tester:test:84 - AUC  : 0.985
2023-10-15 21:40:22.770 | INFO     | tester:test:84 - F1   : 0.8165
2023-10-15 21:40:22.770 | INFO     | tester:test:84 - Acc  : 0.9676
2023-10-15 21:40:22.771 | INFO     | tester:test:84 - Sen  : 0.8249
2023-10-15 21:40:22.772 | INFO     | tester:test:84 - Spe  : 0.9814
2023-10-15 21:40:22.772 | INFO     | tester:test:84 - pre  : 0.8107
2023-10-15 21:40:22.773 | INFO     | tester:test:84 - IOU  : 0.69
2023-10-15 21:40:22.773 | INFO     | tester:test:84 - MCC  : 0.7995
TEST (3) | Loss: 0.2264 | AUC 0.9804 F1 0.7160 Acc 0

In [16]:
# FIVES on FIVES
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/SA_UNet/FIVES/CE_DiceLoss/checkpoint-epoch70.pth'
config_file = "../configs/sa_unet_dicebce.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

# FIVES on DRIVE

main_result(config_file, dataset=drive_dataset, bs=4, weight_path=weight_path)

# FIVES on CHASE

main_result(config_file, dataset=chase_dataset,bs=4, weight_path=weight_path)

  0%|          | 0/50 [00:00<?, ?it/s]

TEST (49) | Loss: 0.1693 | AUC 0.9854 F1 0.8655 Acc 0.9800  Sen 0.9134 Spe 0.9848 Pre 0.8234 IOU 0.7638 MCC 0.8564 |B 0.06 D 0.04 |: 100%|██████████| 50/50 [00:35<00:00,  1.43it/s]
2023-10-15 21:41:37.676 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-10-15 21:41:37.677 | INFO     | tester:test:79 - test time:  0.0622
2023-10-15 21:41:37.677 | INFO     | tester:test:80 -      loss:  0.1693
2023-10-15 21:41:37.678 | INFO     | tester:test:84 - AUC  : 0.9854
2023-10-15 21:41:37.678 | INFO     | tester:test:84 - F1   : 0.8655
2023-10-15 21:41:37.679 | INFO     | tester:test:84 - Acc  : 0.98
2023-10-15 21:41:37.679 | INFO     | tester:test:84 - Sen  : 0.9134
2023-10-15 21:41:37.681 | INFO     | tester:test:84 - Spe  : 0.9848
2023-10-15 21:41:37.681 | INFO     | tester:test:84 - pre  : 0.8234
2023-10-15 21:41:37.682 | INFO     | tester:test:84 - IOU  : 0.7638
2023-10-15 21:41:37.683 | INFO     | tester:test:84 - MCC  : 0.8564
TEST (4) | Loss: 0.2306 | AUC 0.9502 F1 0.7747 

## FR-UNet

In [17]:
# CHASEDB on CHASEDB
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/FR_UNet/CHASE/CE_DiceLoss/checkpoint-epoch80.pth'
config_file = "../config/fr_unet_chase.yaml"


main_result(config_file, dataset=chase_dataset, bs=4, weight_path=weight_path)

# # CHASEDB on DRIVE
main_result(config_file, dataset=drive_dataset, bs=4, weight_path=weight_path)

# CHASEDB on FIVES

main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (1) | Loss: 0.1373 | AUC 0.9900 F1 0.8165 Acc 0.9762  Sen 0.8390 Spe 0.9854 Pre 0.7961 IOU 0.6899 MCC 0.8044 |B 0.76 D 0.41 |: 100%|██████████| 2/2 [00:02<00:00,  1.40s/it]
2023-10-15 21:43:46.164 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-10-15 21:43:46.165 | INFO     | tester:test:79 - test time:  0.7585
2023-10-15 21:43:46.165 | INFO     | tester:test:80 -      loss:  0.1373
2023-10-15 21:43:46.166 | INFO     | tester:test:84 - AUC  : 0.99
2023-10-15 21:43:46.167 | INFO     | tester:test:84 - F1   : 0.8165
2023-10-15 21:43:46.167 | INFO     | tester:test:84 - Acc  : 0.9762
2023-10-15 21:43:46.168 | INFO     | tester:test:84 - Sen  : 0.839
2023-10-15 21:43:46.168 | INFO     | tester:test:84 - Spe  : 0.9854
2023-10-15 21:43:46.169 | INFO     | tester:test:84 - pre  : 0.7961
2023-10-15 21:43:46.169 | INFO     | tester:test:84 - IOU  : 0.6899
2023-10-15 21:43:46.170 | INFO     | tester:test:84 - MCC  : 0.8044
TEST (4) | Loss: 0.2246 | AUC 0.9703 F1 0.7260 Acc 

In [18]:
# DRIVE On DRIVE
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/FR_UNet/DRIVE/CE_DiceLoss/checkpoint-epoch80.pth'
config_file = "../config/fr_unet_drive.yaml"


main_result(config_file, dataset=drive_dataset, bs=4, weight_path=weight_path)

# DRIVE on CHASE
main_result(config_file, dataset=chase_dataset, bs=2, weight_path=weight_path)

# DRIVES on FIVES
main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (4) | Loss: 0.1612 | AUC 0.9819 F1 0.8062 Acc 0.9675  Sen 0.7736 Spe 0.9863 Pre 0.8446 IOU 0.6755 MCC 0.7902 |B 1.01 D 0.74 |: 100%|██████████| 5/5 [00:08<00:00,  1.63s/it]
2023-10-15 21:45:41.221 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-10-15 21:45:41.222 | INFO     | tester:test:79 - test time:  1.0115
2023-10-15 21:45:41.223 | INFO     | tester:test:80 -      loss:  0.1612
2023-10-15 21:45:41.223 | INFO     | tester:test:84 - AUC  : 0.9819
2023-10-15 21:45:41.225 | INFO     | tester:test:84 - F1   : 0.8062
2023-10-15 21:45:41.225 | INFO     | tester:test:84 - Acc  : 0.9675
2023-10-15 21:45:41.226 | INFO     | tester:test:84 - Sen  : 0.7736
2023-10-15 21:45:41.226 | INFO     | tester:test:84 - Spe  : 0.9863
2023-10-15 21:45:41.227 | INFO     | tester:test:84 - pre  : 0.8446
2023-10-15 21:45:41.228 | INFO     | tester:test:84 - IOU  : 0.6755
2023-10-15 21:45:41.228 | INFO     | tester:test:84 - MCC  : 0.7902
TEST (3) | Loss: 0.2991 | AUC 0.9445 F1 0.6140 A

In [21]:
# FIVES on FIVES
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/FR_UNet/FIVES/CE_DiceLoss/checkpoint-epoch70.pth'
config_file = "../configs/fr_unet_cldice.yaml"

main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

# FIVES on DRIVE
main_result(config_file, dataset=drive_dataset, bs=4, weight_path=weight_path)

# FIVES on CHASE

main_result(config_file, dataset=chase_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.0829 | AUC 0.9911 F1 0.9037 Acc 0.9872  Sen 0.8686 Spe 0.9960 Pre 0.9430 IOU 0.8257 MCC 0.8980 |B 0.34 D 0.08 |: 100%|██████████| 50/50 [00:47<00:00,  1.05it/s]
2023-10-15 21:53:04.413 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-10-15 21:53:04.414 | INFO     | tester:test:79 - test time:  0.3405
2023-10-15 21:53:04.414 | INFO     | tester:test:80 -      loss:  0.0829
2023-10-15 21:53:04.415 | INFO     | tester:test:84 - AUC  : 0.9911
2023-10-15 21:53:04.416 | INFO     | tester:test:84 - F1   : 0.9037
2023-10-15 21:53:04.416 | INFO     | tester:test:84 - Acc  : 0.9872
2023-10-15 21:53:04.417 | INFO     | tester:test:84 - Sen  : 0.8686
2023-10-15 21:53:04.417 | INFO     | tester:test:84 - Spe  : 0.996
2023-10-15 21:53:04.418 | INFO     | tester:test:84 - pre  : 0.943
2023-10-15 21:53:04.418 | INFO     | tester:test:84 - IOU  : 0.8257
2023-10-15 21:53:04.418 | INFO     | tester:test:84 - MCC  : 0.898
TEST (4) | Loss: 0.2303 | AUC 0.9222 F1 0.7468 A

## MANet

In [22]:
# CHASEDB on CHASEDB
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/MANet/CHASE/CE_DiceLoss/checkpoint-epoch80.pth'
config_file = "../configs/manet_cldice.yaml"

main_result(config_file, dataset=chase_dataset, bs=4, weight_path=weight_path)

# CHASEDB on FIVES
main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

# CHASEDB on DRIVE
main_result(config_file, dataset=drive_dataset, bs=4, weight_path=weight_path)

TEST (1) | Loss: 0.1470 | AUC 0.9876 F1 0.8056 Acc 0.9758  Sen 0.7970 Spe 0.9878 Pre 0.8146 IOU 0.6745 MCC 0.7928 |B 0.67 D 0.40 |: 100%|██████████| 2/2 [00:02<00:00,  1.29s/it]
2023-10-15 21:53:52.220 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-10-15 21:53:52.221 | INFO     | tester:test:79 - test time:  0.6723
2023-10-15 21:53:52.222 | INFO     | tester:test:80 -      loss:  0.147
2023-10-15 21:53:52.222 | INFO     | tester:test:84 - AUC  : 0.9876
2023-10-15 21:53:52.223 | INFO     | tester:test:84 - F1   : 0.8056
2023-10-15 21:53:52.223 | INFO     | tester:test:84 - Acc  : 0.9758
2023-10-15 21:53:52.224 | INFO     | tester:test:84 - Sen  : 0.797
2023-10-15 21:53:52.224 | INFO     | tester:test:84 - Spe  : 0.9878
2023-10-15 21:53:52.226 | INFO     | tester:test:84 - pre  : 0.8146
2023-10-15 21:53:52.227 | INFO     | tester:test:84 - IOU  : 0.6745
2023-10-15 21:53:52.227 | INFO     | tester:test:84 - MCC  : 0.7928
TEST (49) | Loss: 0.1986 | AUC 0.9806 F1 0.7317 Ac

In [8]:
# DRIVE On DRIVE No model
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/MANet/DRIVE/CE_DiceLoss/checkpoint-epoch80.pth'
config_file = "../configs/manet_cldice.yaml"


main_result(config_file, dataset=drive_dataset, bs=4, weight_path=weight_path)

# DRIVE on CHASEDB

main_result(config_file, dataset=chase_dataset, bs=4, weight_path=weight_path)
#DRIVE on FIVES


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (4) | Loss: 0.1904 | AUC 0.9745 F1 0.7777 Acc 0.9631  Sen 0.7397 Spe 0.9847 Pre 0.8225 IOU 0.6364 MCC 0.7596 |B 1.71 D 0.72 |: 100%|██████████| 5/5 [00:11<00:00,  2.36s/it]
2023-11-28 13:12:04.677 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-11-28 13:12:04.677 | INFO     | tester:test:79 - test time:  1.7145
2023-11-28 13:12:04.678 | INFO     | tester:test:80 -      loss:  0.1904
2023-11-28 13:12:04.678 | INFO     | tester:test:84 - AUC  : 0.9745
2023-11-28 13:12:04.679 | INFO     | tester:test:84 - F1   : 0.7777
2023-11-28 13:12:04.679 | INFO     | tester:test:84 - Acc  : 0.9631
2023-11-28 13:12:04.680 | INFO     | tester:test:84 - Sen  : 0.7397
2023-11-28 13:12:04.680 | INFO     | tester:test:84 - Spe  : 0.9847
2023-11-28 13:12:04.681 | INFO     | tester:test:84 - pre  : 0.8225
2023-11-28 13:12:04.681 | INFO     | tester:test:84 - IOU  : 0.6364
2023-11-28 13:12:04.682 | INFO     | tester:test:84 - MCC  : 0.7596
TEST (1) | Loss: 0.2557 | AUC 0.9602 F1 0.6735 A

In [23]:
# FIVES on FIVES
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/MANet/FIVES/CE_DiceLoss/checkpoint-epoch70.pth'
config_file = "../configs/manet_cldice.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

# FIVES on DRIVE

main_result(config_file, dataset=drive_dataset, bs=4, weight_path=weight_path)

# FIVES on CHASE

main_result(config_file, dataset=chase_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.0817 | AUC 0.9910 F1 0.8997 Acc 0.9864  Sen 0.8753 Spe 0.9947 Pre 0.9261 IOU 0.8186 MCC 0.8929 |B 0.51 D 0.48 |: 100%|██████████| 50/50 [00:54<00:00,  1.10s/it]
2023-10-15 22:01:32.174 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-10-15 22:01:32.175 | INFO     | tester:test:79 - test time:  0.5103
2023-10-15 22:01:32.176 | INFO     | tester:test:80 -      loss:  0.0817
2023-10-15 22:01:32.176 | INFO     | tester:test:84 - AUC  : 0.991
2023-10-15 22:01:32.177 | INFO     | tester:test:84 - F1   : 0.8997
2023-10-15 22:01:32.177 | INFO     | tester:test:84 - Acc  : 0.9864
2023-10-15 22:01:32.177 | INFO     | tester:test:84 - Sen  : 0.8753
2023-10-15 22:01:32.178 | INFO     | tester:test:84 - Spe  : 0.9947
2023-10-15 22:01:32.179 | INFO     | tester:test:84 - pre  : 0.9261
2023-10-15 22:01:32.179 | INFO     | tester:test:84 - IOU  : 0.8186
2023-10-15 22:01:32.179 | INFO     | tester:test:84 - MCC  : 0.8929
TEST (4) | Loss: 0.2074 | AUC 0.9466 F1 0.7764

## UNet

In [24]:
# CHASEDB on CHASEDB
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/Unet/CHASE/CE_DiceLoss/checkpoint-epoch80.pth'
config_file = "../config/unet_chase.yaml"


main_result(config_file, dataset=chase_dataset, bs=4, weight_path=weight_path)

# CHASEDB on DRIVE

main_result(config_file, dataset=drive_dataset, bs=4, weight_path=weight_path)

# CHASEDB on FIVES

main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (1) | Loss: 0.1480 | AUC 0.9877 F1 0.8055 Acc 0.9752  Sen 0.8160 Spe 0.9859 Pre 0.7962 IOU 0.6744 MCC 0.7926 |B 0.46 D 0.43 |: 100%|██████████| 2/2 [00:02<00:00,  1.08s/it]
2023-10-15 22:04:40.458 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-10-15 22:04:40.459 | INFO     | tester:test:79 - test time:  0.4565
2023-10-15 22:04:40.460 | INFO     | tester:test:80 -      loss:  0.148
2023-10-15 22:04:40.460 | INFO     | tester:test:84 - AUC  : 0.9877
2023-10-15 22:04:40.461 | INFO     | tester:test:84 - F1   : 0.8055
2023-10-15 22:04:40.461 | INFO     | tester:test:84 - Acc  : 0.9752
2023-10-15 22:04:40.463 | INFO     | tester:test:84 - Sen  : 0.816
2023-10-15 22:04:40.463 | INFO     | tester:test:84 - Spe  : 0.9859
2023-10-15 22:04:40.464 | INFO     | tester:test:84 - pre  : 0.7962
2023-10-15 22:04:40.464 | INFO     | tester:test:84 - IOU  : 0.6744
2023-10-15 22:04:40.465 | INFO     | tester:test:84 - MCC  : 0.7926
TEST (4) | Loss: 0.2358 | AUC 0.9522 F1 0.7228 Acc

In [25]:
# DRIVE On DRIVE

weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/Unet/DRIVE/CE_DiceLoss/checkpoint-epoch80.pth'
config_file = "../config/unet_drive.yaml"


main_result(config_file, dataset=drive_dataset, bs=4, weight_path=weight_path)
# DRIVE ON CHASE

main_result(config_file, dataset=chase_dataset, bs=4, weight_path=weight_path)
# DRIVE on FIVES

main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

TEST (4) | Loss: 0.1952 | AUC 0.9703 F1 0.7723 Acc 0.9629  Sen 0.7198 Spe 0.9864 Pre 0.8357 IOU 0.6293 MCC 0.7554 |B 0.40 D 0.37 |: 100%|██████████| 5/5 [00:04<00:00,  1.02it/s]
2023-10-15 22:11:13.208 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-10-15 22:11:13.209 | INFO     | tester:test:79 - test time:  0.3958
2023-10-15 22:11:13.209 | INFO     | tester:test:80 -      loss:  0.1952
2023-10-15 22:11:13.210 | INFO     | tester:test:84 - AUC  : 0.9703
2023-10-15 22:11:13.211 | INFO     | tester:test:84 - F1   : 0.7723
2023-10-15 22:11:13.211 | INFO     | tester:test:84 - Acc  : 0.9629
2023-10-15 22:11:13.212 | INFO     | tester:test:84 - Sen  : 0.7198
2023-10-15 22:11:13.212 | INFO     | tester:test:84 - Spe  : 0.9864
2023-10-15 22:11:13.213 | INFO     | tester:test:84 - pre  : 0.8357
2023-10-15 22:11:13.213 | INFO     | tester:test:84 - IOU  : 0.6293
2023-10-15 22:11:13.214 | INFO     | tester:test:84 - MCC  : 0.7554
TEST (1) | Loss: 0.2328 | AUC 0.9671 F1 0.7034 A

In [27]:
# FIVES on FIVES
weight_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/Unet/FIVES/CE_DiceLoss/checkpoint-epoch70.pth'
config_file = "../configs/unet_dicebce.yaml"


main_result(config_file, dataset=fives_dataset, bs=4, weight_path=weight_path)

# FIVES on DRIVE

main_result(config_file, dataset=drive_dataset, bs=4, weight_path=weight_path)
# FIVES on CHASE

main_result(config_file, dataset=chase_dataset, bs=4, weight_path=weight_path)

TEST (49) | Loss: 0.0800 | AUC 0.9914 F1 0.9015 Acc 0.9866  Sen 0.8838 Spe 0.9942 Pre 0.9208 IOU 0.8218 MCC 0.8947 |B 0.08 D 0.05 |: 100%|██████████| 50/50 [00:32<00:00,  1.52it/s]
2023-10-15 22:13:07.905 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-10-15 22:13:07.906 | INFO     | tester:test:79 - test time:  0.0786
2023-10-15 22:13:07.907 | INFO     | tester:test:80 -      loss:  0.08
2023-10-15 22:13:07.907 | INFO     | tester:test:84 - AUC  : 0.9914
2023-10-15 22:13:07.908 | INFO     | tester:test:84 - F1   : 0.9015
2023-10-15 22:13:07.909 | INFO     | tester:test:84 - Acc  : 0.9866
2023-10-15 22:13:07.909 | INFO     | tester:test:84 - Sen  : 0.8838
2023-10-15 22:13:07.909 | INFO     | tester:test:84 - Spe  : 0.9942
2023-10-15 22:13:07.910 | INFO     | tester:test:84 - pre  : 0.9208
2023-10-15 22:13:07.910 | INFO     | tester:test:84 - IOU  : 0.8218
2023-10-15 22:13:07.911 | INFO     | tester:test:84 - MCC  : 0.8947
TEST (4) | Loss: 0.2128 | AUC 0.9398 F1 0.7757 

## Tensorboard session

In [ ]:
from torch.utils import tensorboard

In [ ]:
tensorboard --logdir=/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/WNet/WNet/DiceLoss/ --bind_all